In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install emoji

     |████████████████████████████████| 1.2 MB 14.5 MB/s 
     |████████████████████████████████| 4.0 MB 15.5 MB/s 
     |████████████████████████████████| 895 kB 70.1 MB/s 
     |████████████████████████████████| 596 kB 79.6 MB/s 
     |████████████████████████████████| 6.5 MB 50.0 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 175 kB 14.6 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=1c7db8830ae822aab3d8ada2e330e8640bae8e4fff00472babbc6b7e088b4f4c
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import transformers
import emoji
from transformers import  AdamW, get_linear_schedule_with_warmup
import re
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import sentencepiece
RANDOM_SEED = 69  
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [ ]:
emoji_data = pd.read_csv('/content/drive/MyDrive/Emogen/emogen_mask_70.csv')
emoji_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3071487 entries, 0 to 3071486
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Text           object
 1   Label          object
 2   Unmasked_Text  object
 3   Class_Text     object
dtypes: object(4)
memory usage: 93.7+ MB


In [ ]:
emoji_data.head()

,Text,Label,Unmasked_Text,Class_Text
0,i hesitated to buy a pc and now it’s gone [MASK],😭,i hesitated to buy a pc and now it’s gone 😭 😭,i hesitated to buy a pc and now it’s gone
1,mouloud sghir [MASK] wtf,😭,mouloud sghir 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 😭 wtf,mouloud sghir wtf
2,also nobody gonna stop saying the word himbo b...,😭,also nobody gonna stop saying the word himbo b...,also nobody gonna stop saying the word himbo b...
3,i just wanna know what channel the game gone b...,😭,i just wanna know what channel the game gone b...,i just wanna know what channel the game gone b...
4,lmfaoooooo [MASK] please seek help x,😭,lmfaoooooo 😭 😭 😭 please seek help x,lmfaoooooo please seek help x


In [ ]:
class_names = emoji_data.Label.unique()
values = np.arange(51)
dictionary_emo_val = dict(zip(values, class_names))
dictionary_val_emo = dict(zip(class_names, values))
emoji_updated = emoji_data.replace(dictionary_val_emo)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 64

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class EmojiDataset(Dataset):

  def __init__(self, sentences, targets, tokenizer, max_len):
    self.sentences = sentences
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.sentences)
  
  def __getitem__(self, item):
    review = str(self.sentences[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
      truncation=True
    )

    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = EmojiDataset(
    sentences=df.Class_Text.to_numpy(),
    targets=df.Label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels = 51)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
emoji_updated = emoji_updated.dropna()
df_train, df_test = train_test_split(emoji_updated, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)
df_train.shape, df_val.shape, df_test.shape

((2764338, 4), (153574, 4), (153575, 4))

In [ ]:
BATCH_SIZE = 64
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
#model = SentimentClassifier(len(class_names))
model = model.to(device)
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, eps = 1e-6, weight_decay = .01)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from tqdm.auto import tqdm
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  loop = tqdm(data_loader)
  for d in loop:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask,
      labels = targets
    )

    _, preds = torch.max(outputs.logits, dim=1)
    loss = outputs.loss

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels = targets
      )
      _, preds = torch.max(outputs.logits, dim=1)

      loss = outputs.loss

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
#checkpoint = torch.load('/content/drive/MyDrive/Emogen/class_checkpoint_3_28')
#optimizer = AdamW(model.parameters(), lr=1.5e-5, eps = 1e-6, weight_decay = .01)
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
%%time

best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  ) 

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    model.save_pretrained("/content/drive/MyDrive/Emogen/emogen_base_bert_class")
    tokenizer.save_pretrained("/content/drive/MyDrive/Emogen/emogen_base_bert_class_tokenizer")
    best_accuracy = val_acc

Epoch 1/5
----------


  0%|          | 0/43193 [00:00<?, ?it/s]

Train loss 3.309087787877715 accuracy 0.15559421460038533
Val   loss 3.221982920865218 accuracy 0.17340174769166655

Epoch 2/5
----------


  0%|          | 0/43193 [00:00<?, ?it/s]

Train loss 3.1454756896024922 accuracy 0.18748286208126502
Val   loss 3.193850930929184 accuracy 0.17975699011551433

Epoch 3/5
----------


...

Train loss 3.0363528878293646 accuracy 0.20897733923999162
Val   loss 3.1985512005289394 accuracy 0.181606261476552

Epoch 4/5
----------


  0%|          | 0/43193 [00:00<?, ?it/s]

Train loss 2.941812205087771 accuracy 0.22776158342431352
Val   loss 3.2155000683665276 accuracy 0.18197090653365802

Epoch 5/5
----------


  0%|          | 0/43193 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, '/content/drive/MyDrive/Emogen/class_checkpoint_3_28')

In [ ]:
val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

print(f'Val   loss {val_loss} accuracy {val_acc}')

In [ ]:
model.save_pretrained("emogen_class_small_debertxsmall")
tokenizer.save_pretrained("emogen_class_small_tokenizer")

In [ ]:
model = SentimentClassifier(len(class_names))
model.load_state_dict(torch.load('/content/drive/MyDrive/Emogen/Emogen_test2emojis_smalldataset.bin'))
model = model.to(device)

In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:
      
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels = targets
      )


      outputs = model(
        input_ids=input_ids,
       attention_mask=attention_mask
      )
      _, preds = torch.max(outputs.logits, dim=1)

      predictions.extend(preds)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  real_values = torch.stack(real_values).cpu()
  return predictions, real_values

In [ ]:
y_pred, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names, digits = 4))

              precision    recall  f1-score   support

           😭     0.1471    0.1512    0.1491      3221
           😂     0.1083    0.0917    0.0993      3228
           🤣     0.1151    0.0734    0.0896      3081
           🙃     0.1262    0.0970    0.1097      2968
           😶     0.1294    0.0750    0.0949      3068
           🥺     0.1762    0.2829    0.2172      3079
           😊     0.2310    0.2832    0.2545      3139
           🔥     0.2811    0.3677    0.3186      2940
           😍     0.1794    0.2163    0.1961      3125
           🥰     0.1538    0.1246    0.1376      3123
           😅     0.0902    0.0649    0.0755      3003
           🤪     0.0877    0.0549    0.0676      2895
           😱     0.1695    0.1826    0.1758      2962
           😆     0.1173    0.0769    0.0929      3015
           😇     0.2863    0.1876    0.2267      2996
           🤔     0.1848    0.3106    0.2318      2762
           🤧     0.1653    0.0677    0.0961      3203
           😃     0.1713    

In [ ]:
def test_preds():
  text = builtins.input('Gimme a Sentence:   ') 
  x = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=150,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
      ).to(device)
  _, preds = torch.topk(model(x['input_ids'],x['attention_mask']),k=3,dim= 1)
  return class_names[preds.detach().cpu().numpy()[0]]

In [ ]:
def test_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.topk(outputs,k=3,dim= 1)
      for index, target in enumerate(list(targets)):
          if target in preds[index]:
            correct_predictions += 1
      loss = loss_fn(outputs, targets)

      #correct_predictions += torch.sum(targets in preds)
      losses.append(loss.item())

  return float(correct_predictions) / n_examples, np.mean(losses)

In [ ]:
test_acc, test_loss = test_model(
    model,
    test_data_loader,
    loss_fn, 
    device, 
    len(df_test)
  )